In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets
from ipywidgets import FloatProgress
import mne
from mne.datasets.sleep_physionet.age import fetch_data
from mne.time_frequency import psd_welch

In [141]:
result = []
for i in range(0,10): #2 to test
    if i not in [36, 39, 52, 68, 69, 78, 79]:
        subject = fetch_data(subjects = [i], recording = [1])
        raw_train = mne.io.read_raw_edf(subject[0][0])
        annot_train = mne.read_annotations(subject[0][1])
        raw_train.set_annotations(annot_train, emit_warning=False)
        
        annotation_desc_2_event_id = {'Sleep stage W': 1,
                              'Sleep stage 1': 2,
                              'Sleep stage 2': 3,
                              'Sleep stage 3': 4,
                              'Sleep stage 4': 4,
                              'Sleep stage R': 5}

        # keep last 30-min wake events before sleep and first 30-min wake events after
        # sleep and redefine annotations on raw data
        annot_train.crop(annot_train[1]['onset'] - 30 * 60,
                         annot_train[-2]['onset'] + 30 * 60)
        raw_train.set_annotations(annot_train, emit_warning=False)

        events_train, _ = mne.events_from_annotations(
            raw_train, event_id=annotation_desc_2_event_id, chunk_duration=30.)

        # create a new event_id that unifies stages 3 and 4
        event_id = {'Sleep stage W': 1,
                    'Sleep stage 1': 2,
                    'Sleep stage 2': 3,
                    'Sleep stage 3/4': 4,
                    'Sleep stage R': 5}

        tmax = 30. - 1. / raw_train.info['sfreq']  # tmax in included

        epochs_train = mne.Epochs(raw=raw_train, events=events_train,
                                  event_id=event_id, tmin=0., tmax=tmax, baseline=None)

        index, scalings = ['epoch'], dict(eeg=1e6, mag=1e15, grad=1e13)

        df = epochs_train.to_data_frame(picks=None, scalings=scalings)
        #take first condition value per epoch (is there 1 epoch to condition? not sure)
        df = pd.DataFrame(df.groupby('epoch').first()['condition']).merge(df.groupby('epoch').mean().drop('time', axis = 1), left_index = True, right_index = True, how = 'inner')

        df['patient_id'] = i + 1
        result.append(df)
result = pd.concat(result)

Using default location ~/mne_data for PHYSIONET_SLEEP...
Extracting EDF parameters from C:\Users\baron\mne_data\physionet-sleep-data\SC4001E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
Not setting metadata
841 matching events found
No baseline correction applied
0 projection items activated
Loading data for 841 events and 3000 original time points ...
0 bad epochs dropped
Using default location ~/mne_data for PHYSIONET_SLEEP...
Extracting EDF parameters from C:\Users\baron\mne_data\physionet-sleep-data\SC4011E0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Sleep stage 1', 'Sleep stage 2', 'Sleep stage 3', 'Sleep stage 4', 'Sleep stage R', 'Sleep stage W']
Not setting metadata
Not setting metadata
1103 ma

In [148]:
result[['patient_id','condition', 'EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal',
       'Resp oro-nasal', 'EMG submental', 'Temp rectal', 'Event marker']]

,patient_id,condition,EEG Fpz-Cz,EEG Pz-Oz,EOG horizontal,Resp oro-nasal,EMG submental,Temp rectal,Event marker
epoch,,,,,,,,,
0,1,Sleep stage W,0.018630,-0.570858,2.179144,1.174000e+08,3.422800,3.703405e+07,1970-01-01 00:00:00.854700000
1,1,Sleep stage W,0.847832,-0.256746,0.120571,1.349667e+08,3.471267,3.702903e+07,1970-01-01 00:00:00.842699999
2,1,Sleep stage W,-0.212739,-0.408892,0.886047,1.233000e+08,3.406200,3.703387e+07,1970-01-01 00:00:00.827933333
3,1,Sleep stage W,-0.331894,-0.671820,3.943847,1.277333e+08,3.418800,3.702133e+07,1970-01-01 00:00:00.828699999
4,1,Sleep stage W,0.394721,0.019937,-3.596426,1.288000e+08,3.454933,3.702391e+07,1970-01-01 00:00:00.842666666
...,...,...,...,...,...,...,...,...,...
1127,10,Sleep stage W,-0.456694,0.528267,5.848229,1.609333e+08,4.090800,3.712866e+07,1970-01-01 00:00:01.268066666
1128,10,Sleep stage W,0.223672,0.674795,1.641696,1.295000e+08,4.001667,3.713412e+07,1970-01-01 00:00:01.255066666
1129,10,Sleep stage W,-0.451798,0.805049,3.626905,1.177000e+08,3.954000,3.714003e+07,1970-01-01 00:00:01.250900000
